# Morning

In [1]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import *
import numpy as np
import csv
import os
import time

# ✅ Load YOLO model
model = YOLO('yolov8s.pt')

# ✅ Object classes
class_list = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
    "chair", "couch", "potted plant", "bed", "dining table", "toilet", "TV", "laptop",
    "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

# ✅ Initialize tracker
tracker = Tracker()
cap = cv2.VideoCapture('camp12_morning.MOV')

# ✅ CSV File Setup
csv_filename = "vehicles_summary.csv"

# ✅ Create CSV file if it doesn't exist
if not os.path.exists(csv_filename):
    with open(csv_filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Cars", "Buses", "Trucks", "Motorcycles"])

# ✅ Define reference lines
red_line_y = 360
blue_line_y = 400
offset = 7  # Allow slight variation due to tracking jitter

# ✅ Define vehicle classes to track
vehicle_classes = ["car", "bus", "truck", "motorcycle"]
vehicle_counts = {"car": 0, "bus": 0, "truck": 0, "motorcycle": 0}

# ✅ Store vehicle IDs that crossed the red line first
crossed_red = {}
counter_down = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (1020, 500))

    # ✅ YOLO Detection
    results = model.predict(frame)
    a = results[0].boxes.data.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    # ✅ Store detections
    detections = []
    detected_vehicles = {}

    for _, row in px.iterrows():
        x1, y1, x2, y2 = int(row[0]), int(row[1]), int(row[2]), int(row[3])
        d = int(row[5])
        vehicle_type = class_list[d]

        if vehicle_type in vehicle_classes:
            detections.append([x1, y1, x2, y2])
            detected_vehicles[(x1, y1, x2, y2)] = vehicle_type

    # ✅ Ensure 'detections' is not empty
    bbox_id = tracker.update(np.array(detections)) if detections else []

    # ✅ Process ALL detected vehicles
    for bbox in bbox_id:
        x3, y3, x4, y4, vehicle_id = bbox
        cx, cy = (x3 + x4) // 2, (y3 + y4) // 2

        vehicle_type = detected_vehicles.get((x3, y3, x4, y4), "unknown")

        # ✅ Check if the vehicle crosses the RED line first
        if red_line_y - offset < cy < red_line_y + offset:
            crossed_red[vehicle_id] = (cy, vehicle_type)

        # ✅ Only consider vehicles that crossed red before reaching blue
        if vehicle_id in crossed_red and blue_line_y - offset < cy < blue_line_y + offset:
            _, v_type = crossed_red[vehicle_id]

            # ✅ Draw circle at the blue line for valid vehicles
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"{v_type}-{vehicle_id}", (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

            # ✅ Add to counter only once
            if vehicle_id not in counter_down:
                counter_down.append(vehicle_id)
                vehicle_counts[v_type] += 1

    # ✅ Draw reference lines
    cv2.line(frame, (260, red_line_y), (724, red_line_y), (0, 0, 255), 3)
    cv2.putText(frame, 'red line', (260, red_line_y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.line(frame, (200, blue_line_y), (779, blue_line_y), (255, 0, 0), 3)
    cv2.putText(frame, 'blue line', (200, blue_line_y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # ✅ Display vehicle count
    cv2.putText(frame, f"Cars: {vehicle_counts['car']}  Buses: {vehicle_counts['bus']}  Trucks: {vehicle_counts['truck']}  Motorcycles: {vehicle_counts['motorcycle']}",
                (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow("frames", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# ✅ Log the final vehicle count when the video ends
timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
number_of_lanes=2
# ✅ Calculate light duration using the given formula
light_duration = ((vehicle_counts["car"] * 2) + 
                  (vehicle_counts["motorcycle"] * 1.8) + 
                  ((vehicle_counts["bus"] + vehicle_counts["truck"]) * 3.5)) / number_of_lanes
with open(csv_filename, mode="a", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([timestamp, vehicle_counts["car"], vehicle_counts["bus"], vehicle_counts["truck"], vehicle_counts["motorcycle"], round(light_duration, 2)])

print(f"✅ Final vehicle count saved to CSV: {vehicle_counts}, Light Duration: {round(light_duration, 2)} sec")


cap.release()
cv2.destroyAllWindows()



0: 320x640 11 persons, 2 cars, 1 motorcycle, 2 buss, 324.8ms
Speed: 8.8ms preprocess, 324.8ms inference, 3.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 persons, 1 car, 1 motorcycle, 2 buss, 274.7ms
Speed: 5.7ms preprocess, 274.7ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 persons, 1 car, 2 motorcycles, 1 bus, 256.8ms
Speed: 5.1ms preprocess, 256.8ms inference, 2.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 persons, 1 car, 2 motorcycles, 1 bus, 247.9ms
Speed: 4.8ms preprocess, 247.9ms inference, 2.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 persons, 1 car, 2 motorcycles, 1 bus, 231.3ms
Speed: 4.9ms preprocess, 231.3ms inference, 2.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 persons, 1 car, 1 motorcycle, 1 bus, 227.2ms
Speed: 3.8ms preprocess, 227.2ms inference, 2.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 persons, 2 cars, 1 motorcycle, 1 bus,

KeyboardInterrupt: 

# Rain

In [2]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import *
import numpy as np
import csv
import os
import time

# ✅ Load YOLO model
model = YOLO('yolov8s.pt')

# ✅ Object classes
class_list = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
    "chair", "couch", "potted plant", "bed", "dining table", "toilet", "TV", "laptop",
    "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

# ✅ Initialize tracker
tracker = Tracker()
cap = cv2.VideoCapture('camp12_rain_noon.mp4')

# ✅ CSV File Setup
csv_filename = "vehicles_summary.csv"

# ✅ Create CSV file if it doesn't exist
if not os.path.exists(csv_filename):
    with open(csv_filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Cars", "Buses", "Trucks", "Motorcycles"])

# ✅ Define reference lines
red_line_y = 320
blue_line_y = 350
offset = 7  # Allow slight variation due to tracking jitter

# ✅ Define vehicle classes to track
vehicle_classes = ["car", "bus", "truck", "motorcycle"]
vehicle_counts = {"car": 0, "bus": 0, "truck": 0, "motorcycle": 0}

# ✅ Store vehicle IDs that crossed the red line first
crossed_red = {}
counter_down = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (1020, 500))

    # ✅ YOLO Detection
    results = model.predict(frame)
    a = results[0].boxes.data.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    # ✅ Store detections
    detections = []
    detected_vehicles = {}

    for _, row in px.iterrows():
        x1, y1, x2, y2 = int(row[0]), int(row[1]), int(row[2]), int(row[3])
        d = int(row[5])
        vehicle_type = class_list[d]

        if vehicle_type in vehicle_classes:
            detections.append([x1, y1, x2, y2])
            detected_vehicles[(x1, y1, x2, y2)] = vehicle_type

    # ✅ Ensure 'detections' is not empty
    bbox_id = tracker.update(np.array(detections)) if detections else []

    # ✅ Process ALL detected vehicles
    for bbox in bbox_id:
        x3, y3, x4, y4, vehicle_id = bbox
        cx, cy = (x3 + x4) // 2, (y3 + y4) // 2

        vehicle_type = detected_vehicles.get((x3, y3, x4, y4), "unknown")

        # ✅ Check if the vehicle crosses the RED line first
        if red_line_y - offset < cy < red_line_y + offset:
            crossed_red[vehicle_id] = (cy, vehicle_type)

        # ✅ Only consider vehicles that crossed red before reaching blue
        if vehicle_id in crossed_red and blue_line_y - offset < cy < blue_line_y + offset:
            _, v_type = crossed_red[vehicle_id]

            # ✅ Draw circle at the blue line for valid vehicles
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"{v_type}-{vehicle_id}", (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

            # ✅ Add to counter only once
            if vehicle_id not in counter_down:
                counter_down.append(vehicle_id)
                vehicle_counts[v_type] += 1

    # ✅ Draw reference lines
    cv2.line(frame, (180, red_line_y), (950, red_line_y), (0, 0, 255), 3)
    cv2.putText(frame, 'red line', (180, red_line_y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.line(frame, (150, blue_line_y), (950, blue_line_y), (255, 0, 0), 3)
    cv2.putText(frame, 'blue line', (150, blue_line_y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # ✅ Display vehicle count
    cv2.putText(frame, f"Cars: {vehicle_counts['car']}  Buses: {vehicle_counts['bus']}  Trucks: {vehicle_counts['truck']}  Motorcycles: {vehicle_counts['motorcycle']}",
                (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow("frames", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# ✅ Log the final vehicle count when the video ends
timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
number_of_lanes=2
# ✅ Calculate light duration using the given formula
light_duration = ((vehicle_counts["car"] * 2) + 
                  (vehicle_counts["motorcycle"] * 1.8) + 
                  ((vehicle_counts["bus"] + vehicle_counts["truck"]) * 3.5)) / number_of_lanes
with open(csv_filename, mode="a", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([timestamp, vehicle_counts["car"], vehicle_counts["bus"], vehicle_counts["truck"], vehicle_counts["motorcycle"], round(light_duration, 2)])

print(f"✅ Final vehicle count saved to CSV: {vehicle_counts}, Light Duration: {round(light_duration, 2)} sec")


cap.release()
cv2.destroyAllWindows()



0: 320x640 3 persons, 1 car, 2 motorcycles, 2 trucks, 2 umbrellas, 274.9ms
Speed: 5.2ms preprocess, 274.9ms inference, 2.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 3 cars, 2 motorcycles, 3 trucks, 2 umbrellas, 272.7ms
Speed: 4.4ms preprocess, 272.7ms inference, 3.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 1 car, 2 motorcycles, 3 trucks, 2 umbrellas, 307.4ms
Speed: 6.2ms preprocess, 307.4ms inference, 2.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 3 motorcycles, 3 trucks, 2 umbrellas, 302.0ms
Speed: 11.6ms preprocess, 302.0ms inference, 2.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 2 cars, 3 motorcycles, 2 trucks, 2 umbrellas, 253.4ms
Speed: 6.4ms preprocess, 253.4ms inference, 2.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 2 cars, 3 motorcycles, 2 trucks, 2 umbrellas, 260.0ms
Speed: 4.7ms preprocess, 260.0ms inference, 2.5ms postpro

KeyboardInterrupt: 

# Noon

In [3]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import *
import numpy as np
import csv
import os
import time

# ✅ Load YOLO model
model = YOLO('yolov8s.pt')

# ✅ Object classes
class_list = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
    "chair", "couch", "potted plant", "bed", "dining table", "toilet", "TV", "laptop",
    "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

# ✅ Initialize tracker
tracker = Tracker()
cap = cv2.VideoCapture('IMG_8981.MOV')

# ✅ CSV File Setup
csv_filename = "vehicles_summary.csv"

# ✅ Create CSV file if it doesn't exist
if not os.path.exists(csv_filename):
    with open(csv_filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Cars", "Buses", "Trucks", "Motorcycles"])

# ✅ Define reference lines
red_line_y = 250
blue_line_y = 340
offset = 7  # Allow slight variation due to tracking jitter

# ✅ Define vehicle classes to track
vehicle_classes = ["car", "bus", "truck", "motorcycle"]
vehicle_counts = {"car": 0, "bus": 0, "truck": 0, "motorcycle": 0}

# ✅ Store vehicle IDs that crossed the red line first
crossed_red = {}
counter_down = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (1020, 500))

    # ✅ YOLO Detection
    results = model.predict(frame)
    a = results[0].boxes.data.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    # ✅ Store detections
    detections = []
    detected_vehicles = {}

    for _, row in px.iterrows():
        x1, y1, x2, y2 = int(row[0]), int(row[1]), int(row[2]), int(row[3])
        d = int(row[5])
        vehicle_type = class_list[d]

        if vehicle_type in vehicle_classes:
            detections.append([x1, y1, x2, y2])
            detected_vehicles[(x1, y1, x2, y2)] = vehicle_type

    # ✅ Ensure 'detections' is not empty
    bbox_id = tracker.update(np.array(detections)) if detections else []

    # ✅ Process ALL detected vehicles
    for bbox in bbox_id:
        x3, y3, x4, y4, vehicle_id = bbox
        cx, cy = (x3 + x4) // 2, (y3 + y4) // 2

        vehicle_type = detected_vehicles.get((x3, y3, x4, y4), "unknown")

        # ✅ Check if the vehicle crosses the RED line first
        if red_line_y - offset < cy < red_line_y + offset:
            crossed_red[vehicle_id] = (cy, vehicle_type)

        # ✅ Only consider vehicles that crossed red before reaching blue
        if vehicle_id in crossed_red and blue_line_y - offset < cy < blue_line_y + offset:
            _, v_type = crossed_red[vehicle_id]

            # ✅ Draw circle at the blue line for valid vehicles
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"{v_type}-{vehicle_id}", (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

            # ✅ Add to counter only once
            if vehicle_id not in counter_down:
                counter_down.append(vehicle_id)
                vehicle_counts[v_type] += 1

    # ✅ Draw reference lines
    cv2.line(frame, (550, red_line_y), (880, red_line_y), (0, 0, 255), 3)
    cv2.putText(frame, 'red line', (550, red_line_y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.line(frame, (200, blue_line_y), (900, blue_line_y), (255, 0, 0), 3)
    cv2.putText(frame, 'blue line', (200, blue_line_y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # ✅ Display vehicle count
    cv2.putText(frame, f"Cars: {vehicle_counts['car']}  Buses: {vehicle_counts['bus']}  Trucks: {vehicle_counts['truck']}  Motorcycles: {vehicle_counts['motorcycle']}",
                (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow("frames", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# ✅ Log the final vehicle count when the video ends
timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
number_of_lanes=2
# ✅ Calculate light duration using the given formula
light_duration = ((vehicle_counts["car"] * 2) + 
                  (vehicle_counts["motorcycle"] * 1.8) + 
                  ((vehicle_counts["bus"] + vehicle_counts["truck"]) * 3.5)) / number_of_lanes
with open(csv_filename, mode="a", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([timestamp, vehicle_counts["car"], vehicle_counts["bus"], vehicle_counts["truck"], vehicle_counts["motorcycle"], round(light_duration, 2)])

print(f"✅ Final vehicle count saved to CSV: {vehicle_counts}, Light Duration: {round(light_duration, 2)} sec")


cap.release()
cv2.destroyAllWindows()



0: 320x640 7 persons, 10 cars, 5 motorcycles, 1 truck, 208.7ms
Speed: 6.1ms preprocess, 208.7ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 persons, 9 cars, 6 motorcycles, 1 truck, 241.4ms
Speed: 4.1ms preprocess, 241.4ms inference, 2.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 persons, 9 cars, 5 motorcycles, 1 truck, 221.9ms
Speed: 4.7ms preprocess, 221.9ms inference, 2.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 persons, 9 cars, 5 motorcycles, 1 truck, 1 backpack, 232.1ms
Speed: 3.3ms preprocess, 232.1ms inference, 2.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 persons, 8 cars, 6 motorcycles, 1 truck, 225.5ms
Speed: 76.3ms preprocess, 225.5ms inference, 2.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 persons, 9 cars, 7 motorcycles, 1 truck, 219.6ms
Speed: 5.0ms preprocess, 219.6ms inference, 2.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 person

KeyboardInterrupt: 

# Evening

In [4]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import *
import numpy as np
import csv
import os
import time

# ✅ Load YOLO model
model = YOLO('yolov8s.pt')

# ✅ Object classes
class_list = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
    "chair", "couch", "potted plant", "bed", "dining table", "toilet", "TV", "laptop",
    "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

# ✅ Initialize tracker
tracker = Tracker()
cap = cv2.VideoCapture('camp12_eve.mp4')

# ✅ CSV File Setup
csv_filename = "vehicles_summary.csv"

# ✅ Create CSV file if it doesn't exist
if not os.path.exists(csv_filename):
    with open(csv_filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Cars", "Buses", "Trucks", "Motorcycles"])

# ✅ Define reference lines
red_line_y = 355
blue_line_y = 375
offset = 7  # Allow slight variation due to tracking jitter

# ✅ Define vehicle classes to track
vehicle_classes = ["car", "bus", "truck", "motorcycle"]
vehicle_counts = {"car": 0, "bus": 0, "truck": 0, "motorcycle": 0}

# ✅ Store vehicle IDs that crossed the red line first
crossed_red = {}
counter_down = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (1020, 500))

    # ✅ YOLO Detection
    results = model.predict(frame)
    a = results[0].boxes.data.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    # ✅ Store detections
    detections = []
    detected_vehicles = {}

    for _, row in px.iterrows():
        x1, y1, x2, y2 = int(row[0]), int(row[1]), int(row[2]), int(row[3])
        d = int(row[5])
        vehicle_type = class_list[d]

        if vehicle_type in vehicle_classes:
            detections.append([x1, y1, x2, y2])
            detected_vehicles[(x1, y1, x2, y2)] = vehicle_type

    # ✅ Ensure 'detections' is not empty
    bbox_id = tracker.update(np.array(detections)) if detections else []

    # ✅ Process ALL detected vehicles
    for bbox in bbox_id:
        x3, y3, x4, y4, vehicle_id = bbox
        cx, cy = (x3 + x4) // 2, (y3 + y4) // 2

        vehicle_type = detected_vehicles.get((x3, y3, x4, y4), "unknown")

        # ✅ Check if the vehicle crosses the RED line first
        if red_line_y - offset < cy < red_line_y + offset:
            crossed_red[vehicle_id] = (cy, vehicle_type)

        # ✅ Only consider vehicles that crossed red before reaching blue
        if vehicle_id in crossed_red and blue_line_y - offset < cy < blue_line_y + offset:
            _, v_type = crossed_red[vehicle_id]

            # ✅ Draw circle at the blue line for valid vehicles
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"{v_type}-{vehicle_id}", (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

            # ✅ Add to counter only once
            if vehicle_id not in counter_down:
                counter_down.append(vehicle_id)
                vehicle_counts[v_type] += 1

    # ✅ Draw reference lines
    cv2.line(frame, (260, red_line_y), (724, red_line_y), (0, 0, 255), 3)
    cv2.putText(frame, 'red line', (260, red_line_y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.line(frame, (200, blue_line_y), (779, blue_line_y), (255, 0, 0), 3)
    cv2.putText(frame, 'blue line', (200, blue_line_y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # ✅ Display vehicle count
    cv2.putText(frame, f"Cars: {vehicle_counts['car']}  Buses: {vehicle_counts['bus']}  Trucks: {vehicle_counts['truck']}  Motorcycles: {vehicle_counts['motorcycle']}",
                (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow("frames", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# ✅ Log the final vehicle count when the video ends
timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
number_of_lanes=2
# ✅ Calculate light duration using the given formula
light_duration = ((vehicle_counts["car"] * 2) + 
                  (vehicle_counts["motorcycle"] * 1.8) + 
                  ((vehicle_counts["bus"] + vehicle_counts["truck"]) * 3.5)) / number_of_lanes
with open(csv_filename, mode="a", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([timestamp, vehicle_counts["car"], vehicle_counts["bus"], vehicle_counts["truck"], vehicle_counts["motorcycle"], round(light_duration, 2)])

print(f"✅ Final vehicle count saved to CSV: {vehicle_counts}, Light Duration: {round(light_duration, 2)} sec")


cap.release()
cv2.destroyAllWindows()



0: 320x640 9 persons, 4 cars, 2 motorcycles, 1 truck, 247.3ms
Speed: 4.6ms preprocess, 247.3ms inference, 2.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 persons, 4 cars, 1 motorcycle, 2 trucks, 247.0ms
Speed: 3.8ms preprocess, 247.0ms inference, 2.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 persons, 4 cars, 1 motorcycle, 260.7ms
Speed: 4.9ms preprocess, 260.7ms inference, 2.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 persons, 4 cars, 1 motorcycle, 1 truck, 262.6ms
Speed: 6.5ms preprocess, 262.6ms inference, 3.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 11 persons, 3 cars, 1 motorcycle, 1 truck, 255.7ms
Speed: 5.3ms preprocess, 255.7ms inference, 3.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 persons, 4 cars, 1 motorcycle, 1 truck, 247.8ms
Speed: 5.8ms preprocess, 247.8ms inference, 2.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 persons, 4 cars, 1 motorcycle,

KeyboardInterrupt: 